### Expand application

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_metric
import torch.nn as nn
import torch
from tqdm.auto import tqdm
import random
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from common import load_jsonl, save_jsonl

from transformers import (
    AdamW,
    get_scheduler,
    BertTokenizer,
    AutoTokenizer,
    AutoModelForSequenceClassification
)

In [3]:
class NLIDataset(Dataset):
    def __init__(self, data_list, max_length=512, model_name="bert-base-multilingual-cased"):
        self.d_list = data_list
        self.len = len(self.d_list)
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.label2index = {
            'SPAM': 0,
            'EDM': 1,
            'HAM': 2, 
            'NOTE': 3, 
            'HACK': 4, 
        }

    def __getitem__(self, index):
        data = self.d_list[index]
        context = data['context']
        label = data['label']
        
        processed_sample = dict()
        processed_sample['labels'] = torch.tensor(self.label2index[label])
        tokenized_input = self.tokenizer(context,
                                         max_length=self.max_length,
                                         padding='max_length', 
                                         truncation=True,
                                         return_tensors="pt")
        
        input_items = {key: val.squeeze() for key, val in tokenized_input.items()}
        processed_sample.update(input_items)
        return processed_sample

    def __len__(self):
        return self.len

## load model

In [4]:
model = AutoModelForSequenceClassification.from_pretrained('model/notice_v3_epoch_5', num_labels=5) 
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")  

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if torch.cuda.device_count() >1:
    model = nn.DataParallel(model,device_ids=[0,1])
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

## create file

In [5]:
test_list = load_jsonl('datasets/test_v2.jsonl')
test_dataset = NLIDataset(test_list)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=512)

Load Jsonl: datasets/test_v2.jsonl


34778it [00:01, 18828.11it/s]


In [6]:
model.eval()
predictions = []
references = []
num_steps = len(test_dataloader)
progress_bar = tqdm(range(num_steps))

with torch.no_grad():
    for batch_index, batch_dict in enumerate(test_dataloader):
        input_items = {key: val.to(device) for key, val in batch_dict.items()}
        outputs = model(**input_items)

        predictions += outputs.logits.argmax(dim=-1).tolist()
        references += batch_dict['labels'].tolist()
        progress_bar.update(1)

accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions,average='macro')
print('acc: ', accuracy)
print('f1: ', f1)

  0%|          | 0/68 [00:00<?, ?it/s]

acc:  0.9951981137500718
f1:  0.9814823947571488


In [7]:
print(classification_report(references, predictions))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3670
           1       1.00      1.00      1.00     21985
           2       1.00      1.00      1.00      6776
           3       1.00      1.00      1.00      2225
           4       0.93      0.94      0.93       122

    accuracy                           1.00     34778
   macro avg       0.98      0.98      0.98     34778
weighted avg       1.00      1.00      1.00     34778



In [31]:
import pandas as pd

error_list = []
error_context = []
reference_label_text = []
reference_label_class = []
predict_label = []
output = pd.DataFrame()
class_number = {'SPAM':0, 'EDM':1, 'HAM':2, 'NOTE': 3, 'HACK': 4}

choose_class = 4

for num, i in enumerate(references):
    # if i != predictions[num]:
    if i == choose_class:
        error_list.append(num)
        if test_list[num]['context'] == '':
            # print(num)
            error_context.append('no')
        else:
            error_context.append(test_list[num]['context'])
        
        reference_label_text.append(test_list[num]['label'])
        reference_label_class.append(i)
        predict_label.append(predictions[num])
                
        # print(test_list[num])
        # print("GT : ", references[num])
        # print(predictions[num])
        # break
        # error_context = []
        # predict_label = []
        # reference_label = []
        
print(len(error_list))
# print(error_list)

output['index'] = error_list
output['pred_label'] = predict_label
output['ref_class'] = reference_label_class
output['ref'] = reference_label_text
output['context'] = error_context

# output.to_csv('list_classifier_error.csv', encoding = 'utf-8-sig')
output.to_csv('token/right_%s.csv'%(list(class_number.keys())[list(class_number.values()).index(int(choose_class))]), encoding = 'utf-8-sig')


122


In [36]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2, 'NOTE': 3, 'HACK': 4}

choose_class = 0

model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('token/right_%s.csv'
                      %(list(class_number.keys())[list(class_number.values()).index(int(choose_class))]))[:1000]    # change the value up to the colab limit
    # rows = csv.reader(csvfile)
progress_bar = tqdm(range(len(csvfile['context'])))


for i, token_list in enumerate(csvfile['context']):
    # print(csvfile['index'][i])
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []

    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == choose_class:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")
    progress_bar.update(1)

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))
        

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'管理': 4275, '項目': 3052, '研發': 2656, '企業': 2205, '員工': 1984, '中': 1588, '工作': 1582, '公司': 1527, '分析': 1455, 'UrlText': 1204, '風險': 1144, '經理': 1099, '產品': 1093, 'emailAddress': 938, '方法': 902, '團隊': 901, '控制': 872, '勞動': 750, '成本': 738, '培訓': 715, 'n': 686, '組織': 665, '諮詢': 663, '需求': 617, '溝通': 615, '講師': 606, '激勵': 594, '績效': 590, '人員': 576, '質量': 555, '關係': 550, '新': 545, '技術': 543, '設計': 534, '銷售': 527, 'c': 509, '籌劃': 508, '標準': 499, '進行': 491, '病假': 489, '制定': 480, '能力': 465, '上海': 460, '勞動合同': 457, '過程': 445, 'email': 440, '離職': 438, '工傷': 438, 'ng': 429, '成功': 425, 'u': 422, '手段': 396, '部門': 395, '方案': 385, '模式': 384, '經驗': 383, '勞動者': 382, '採購': 371, '討論': 370, '深圳': 368, '個稅': 368, '產品開發': 359, '演練': 350, '業務': 350, '元': 347, '中國': 344, '原則': 343, '人才': 335, 'make': 332, 'one': 332, '角色': 332, '北京': 331, 'get': 330, '崗位': 329, '學習': 328, '微信': 327, '做': 326, '影響': 326, '實踐': 326, '提升': 325, '開發': 323, '發展': 323, '郵': 321, '箱': 321, '經營': 320, '分享': 319, '體系': 319, '模型': 316, 

In [37]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2, 'NOTE': 3, 'HACK': 4}

choose_class = 1

model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('token/right_%s.csv'
                      %(list(class_number.keys())[list(class_number.values()).index(int(choose_class))]))[:1000]    # change the value up to the colab limit
    # rows = csv.reader(csvfile)
progress_bar = tqdm(range(len(csvfile['context'])))


for i, token_list in enumerate(csvfile['context']):
    # print(csvfile['index'][i])
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []

    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == choose_class:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")
    progress_bar.update(1)

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))
        

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'活動': 1017, '訂閱': 987, '線上': 975, '課程': 825, '內容': 653, 'NT': 543, '時間': 520, '優惠價': 495, '電子報': 483, '優惠': 440, '中心': 424, '前往': 398, '市場': 387, '為準': 367, '系統': 333, '點選': 309, '消費': 280, '顯示': 247, '連結': 232, '按此': 230, '醫療': 229, '特惠價': 225, '下載': 220, '指定': 212, '版權所有': 204, '規格': 198, '簡訊': 182, '好禮': 175, '隱私權': 170, '各項': 168, '培訓班': 164, '廣告': 159, '不想': 157, '申請': 154, '金融': 154, '免費': 152, '經濟': 146, '透過': 136, '百分率': 132, '資格': 131, '詐騙': 123, '無線': 123, '精選': 121, '遠傳': 120, '帳戶': 117, 'Copyright': 117, '電動車': 112, '折扣': 111, 'Unsubscribe': 108, 'Subscribe': 107, '購物網': 104, '政策': 101, '報導': 97, '新竹': 96, '介紹': 93, '新聞': 93, '主辦單位': 90, '查詢': 87, '看不到': 87, '商業': 87, '實驗室': 84, '商機': 84, '繳費': 81, '家庭': 80, '重點': 78, '折價券': 77, '條款': 75, '財務': 75, '結構': 74, '環保': 74, '領域': 74, '查看': 73, '不便': 71, '內部': 69, '瀏覽器': 67, '超過': 65, '經濟部': 64, '視訊': 64, '遠距': 63, '研討會': 63, '基金會': 61, '流行': 60, '適合': 59, '經濟組': 59, '推出': 57, '法規': 56, '著作權': 54, '遊戲': 54, '申辦': 53, '提醒您': 53, '

In [38]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2, 'NOTE': 3, 'HACK': 4}

choose_class = 2

model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('token/right_%s.csv'
                      %(list(class_number.keys())[list(class_number.values()).index(int(choose_class))]))[:1000]    # change the value up to the colab limit
    # rows = csv.reader(csvfile)
progress_bar = tqdm(range(len(csvfile['context'])))


for i, token_list in enumerate(csvfile['context']):
    # print(csvfile['index'][i])
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []

    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == choose_class:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")
    progress_bar.update(1)

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))
        

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'銀行': 1227, '通知': 1211, '交易': 941, '本行': 923, '類別': 860, '股份': 806, '資訊': 779, '金額': 736, '郵件': 716, '回覆': 638, '資料': 594, '客戶': 592, '登入': 559, '臺中市': 544, '付款': 541, '收件人': 538, '狀態': 507, '待遇': 501, '自動': 450, '信用卡': 429, '彰化縣': 398, '代碼': 393, '使用者': 388, '客服': 387, '寫': 361, '履歷': 360, '製造業': 352, '寄件人': 350, '設定': 344, '執行': 333, '商店': 333, '修改': 332, '收到': 328, '學歷': 305, '數量': 301, '職稱': 297, '居住地': 293, '訊息': 282, '情形': 279, 'If': 273, '帳單': 269, '謝謝': 246, '繳款': 243, '帳號': 236, '庫存量': 236, '商品名稱': 227, '累計': 225, '新北市': 224, '商品編號': 223, '個人資料': 222, '備註': 220, '行政': 219, '信用': 211, '現金': 204, '退回': 204, '臺北市': 203, '號': 201, '感謝': 199, '檔案': 195, '轉帳': 191, '臺北': 187, '寄發': 184, '新臺幣': 180, '證照': 176, '手續費': 169, '待業': 164, 'contained': 159, '持有': 157, '費用': 152, '請款': 148, '回饋': 147, '科技股份': 145, '扣款': 144, '並請': 142, '狀況': 136, '臺語': 130, '審核': 129, '業務人員': 129, '顧客': 127, '變更': 127, '高雄市': 126, '機構': 125, '編號': 125, '列印': 124, 'Any': 122, '網頁': 121, '機臺': 120, '商業銀行': 11

In [39]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2, 'NOTE': 3, 'HACK': 4}

choose_class = 3

model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('token/right_%s.csv'
                      %(list(class_number.keys())[list(class_number.values()).index(int(choose_class))]))    # change the value up to the colab limit
    # rows = csv.reader(csvfile)
progress_bar = tqdm(range(len(csvfile['context'])))


for i, token_list in enumerate(csvfile['context']):
    # print(csvfile['index'][i])
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []

    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == choose_class:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")
    progress_bar.update(1)

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))
        

  0%|          | 0/2225 [00:00<?, ?it/s]

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'EyeCloud': 5456, 'eyecloud': 486, '正式版': 413, '已過': 364, '偵測': 162, '科技': 129, 'RMA': 88, '故障': 80, '維修': 54, '終止': 54, '斷線': 24, '測試': 22, '未指定': 21, '發佈者': 20, '臺灣': 18, '檢修': 18, 'None': 13, '檢查': 12, 'Eyecloud': 7, '失效': 6, '最新版': 6, '民航局': 6, '浸水': 6, '施工': 5, '電腦科技': 5, '詳細]': 5, '不行': 4, '水利署': 4, '有沒有': 4, '不通': 4, '破損': 4, '目前': 3, '生物科技': 3, '安危': 2, '低雲影響': 2, '電子科技': 2, '偵測到': 2, '漏水': 2, '損壞': 1, '科技實業': 1, '授權': 1, '舊有': 1, '搶修': 1, '環境保護局': 1, '降雨': 1, '及嘉義': 1, '鐵局': 1, '停電': 1}


In [34]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2, 'NOTE': 3, 'HACK': 4}

choose_class = 4

model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('token/right_%s.csv'
                      %(list(class_number.keys())[list(class_number.values()).index(int(choose_class))]))[:1000]    # change the value up to the colab limit
    # rows = csv.reader(csvfile)
progress_bar = tqdm(range(len(csvfile['context'])))


for i, token_list in enumerate(csvfile['context']):
    # print(csvfile['index'][i])
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []

    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == choose_class:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")
    progress_bar.update(1)

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))
        

  0%|          | 0/122 [00:00<?, ?it/s]

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'regard': 53, 'xlsm': 16, 'adjunto': 10, 'regards': 5, 'kindly': 4, 'thanks': 3, 'zip': 3, 'saludarle': 2, 'hoy': 2, 'Zip': 2, 'yesterday': 2, 'herewith': 1, 'adjuntos': 1, 'nuestra': 1, 'correspondientes': 1, '告訴您': 1, 'ofrecemos': 1, 'quedamos': 1, 'estamos': 1, 'Sra': 1, 'Adjunto': 1}


## notice data

In [15]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2, 'NOT':3, 'HACKER':4}
model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('test_data/notice.csv')    # change the value up to the colab limit
# print(csvfile['context'])
    # rows = csv.reader(csvfile)
for i, token_list in enumerate(csvfile['context']):
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []
    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    # print(predict_list)
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == 3:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{}


## example token

In [6]:
# token predict

edm_1=['新 登入 資訊 確認 FedEx 帳號  XXXXX4045 Alice 感謝 支持 FedEx 登入 整合 計畫 UrlText 登入 資訊 成功 登入 FedEx 酬賓 廣場  平臺 這讓 得以 UrlText 用戶 識別碼  FedEx 酬賓 廣場 安排 管理 到府 取件 管理 線上 帳單 視 查詢 特定 運送 時間 登入 FedEx 酬賓 廣場 查看 紅利 積分 回饋 獎品 更多 功能 如需 進一步 UrlText 帳號 登入 FedEx 酬賓 廣場 網站 請參閱  保護 帳號 顯示  碼 FedEx 帳號  碼  UrlText 登入 帳號 註冊 FedEx 酬賓 廣場 FedEx 帳號 受 FedEx 酬賓 廣場 活動 條款 細則 關注 FEDEX 活動 訊息 通知 發送至 emailAddress FedEx 不定期 寄發 活動 訊息 至此 信箱 取消 訂閱 更新 資料 如本 電子郵件 涉及 個人資料 無意 接受 類似 訊息 按此 回覆 妥善處理 荷蘭商 聯邦快遞 國際 股份 有限公司 臺灣 分公司  臺北市 中山北路 二段  號  樓 emojiPic  FedEx 本信件 受 美國 國際 著作權 商標法 保護 參考 隱私權 聲明 保留 權利 瀏覽 網路版']
edm_2=['xem b ng tr nh duy dear qu kh ch h ng sau th gian ch n novaworld l p n novaland ph tri n ch nh th c c tri n khai v r nhi u ch ng tr nh u h p n nh cho qu kh ch h ng v nh u c h sinh c ng nh u n hi u qu nh  qu kh ch h ng quan n vui l ng li n h bpkd qua n tho     ho c l th']
edm_3=['export target africa market max lewis 09th power energy africa  october   carnivore expo ground nairobi kenya gateway east africa power energy market ding zing chemical product co ltd dear sir east africa invite come explore vast  million consumer market mega']
edm_4=['米思米 官網 首頁 WOS 網上 報價 訂購 賬戶 目錄 申請 忘記 密碼 煩惱 懂 想要 解決 乾貨 滿滿的 知識點 咯  批量 導入 商品 清單 採購 輕鬆 便捷 告別 手動 輸入 高效 採購 時代 終於 訂購 大批量 商品 一份 採購 BOM 導入 上傳 即可  商品 庫存']
edm_5=['special enrollment period end soon advertisement insurance carrier agent broker affiliate plan endorsed government entity agency connect consumer insurance carrier affiliate  main street suite  el segundo ca  emojipic health plan america  unsubscribeview webpage']
edm_6=['dear user update database e mail account also upgrade user effect delete unused e mail account create space new account ensure experience service disruption period need click customer support state kindly customer support validate upgrade mailbox quota also']
edm_7=['limited time offer  prefer receive future email unsubscribe  e 21st street los angeles ca ']
edm_8=['image click jfd training date come  february  february date imca diver medic technician refresher fully book date  feb   feb  find urltext classroom course combine dive rov system audit assurance awareness date  feb   feb  find urltext online course march date imca']
edm_9=['alleviate neck pain home neckmassager must easy solution chronic neck relaxation problem even though technology complex advance simple use save lot pain neckmassager design make feel good possible sit new smoke people modern work force day suffer unnecessary headache due extremely stiff neck neckmassagerhas medical professional go crazy effectiveness problem continue read key feature neckmassager neck pain headache simply disappear  different massage method say goodbye stress worry balance nervous system improve oxygen supply tissue cell nutrition relieve joint pain muscle tension increase blood circulation click neck massager  free shipping like update problem pleaseunsubscribe  clinton street little rock ar  79d 364db television show u take lot emotional issue first two season least death husband father jack season popular drama series ing another controversial important topic obesity fertility u follow life five family include kate woman obese face weight relate stigma throughout initial episode season kate husband visit fertility specialist discus option undergo vitro fertilization ivf struggle overcome emotionally devastate miscarriage season two couple hope ivf help kate pregnant fertility specialist initially refuse take kate patient e embryo say']
edm_10=['親愛的 富邦 信用卡 客戶 感謝 本行 信用卡 持有 信用卡 到期 末  碼  行將 次 掛號 郵寄 方式 寄送 新卡 留存 本行 通訊 地址 臺北市 中山區 長 變更 地址 請於    前 本行 客服 專線 本行 保留 最終 審核 續卡 各項 條件 權利 致電 本行 客服 專線    將有 專員 竭誠 服務 臺北 富邦 銀行 敬啟']


SPAM_1=['chang natapong adul thaebtong thai oil clean fuel project']         
SPAM_2=['Dear Mr Ms 簡瑜瑩 小姐 If message displayed properly please click']
SPAM_3=['下載 達美樂 APP 登入 會員 訂餐 訂單 狀態 達美樂 社? 不定期 好康 送給 Line FB Instagram 封信 系統 發出 信件 回覆 產品 實物 優惠 合併 達美樂 保有 調整 活動 辦法 優惠 內容 權利 取消 訂閱']
SPAM_4=['湘喻 文樂 數學  班 中 張貼 預先 排程 作業 新 作業 截止 日期     課堂 進度 繳交 請於 隔天 中午   前 上傳 方喬飛 p1  潘潔琳 p37  蘇子 琦 p1  孟巧 璇 p99  開啟 張貼者 陳 採靈 張貼 日期   Google LLC  Amphitheatre Parkway Mountain View CA  USA']
SPAM_5=['want change receive email update preference unsubscribe list']
SPAM_6=['read online newsletter february  free subscription qf plus option face attendance new work location feature timetec ta timetec ban jailbroken root phone start timetec ta app timetec renew iso   certification separate contractor normal visitor timetec vms change']
SPAM_7=['航 eMall 哩程 折抵 購物金 超值 公開 父親節 禮物 eMall 買 超優 華航 eMall 哩程 折抵 購物金   折抵 NT  元 每樣 商品 折抵 哩程 睡著 詳情 參考 哩程 折抵 活動 辦法 華夏 會員 享有 更多 優惠 新增 全 哩程 兌換 優惠 館 原價  折起 幫 省 荷包']
SPAM_8=['votre bien tre notre priorit si ce message ne affiche pa correctement cliquez ici pour l ouvrir dans votre navigateur si vous ne souhaitez plus recevoir email de la part des laboratoires olliscience cliquez ici anti ge fin de l offre de lancement ce soir ch lectrice j des']
SPAM_9=['湘喻 潘俞 岑在 新 來義 數學  班 中 張貼 新 公告 新 公告 上課 上線 時間 李喬     高郡成     馬育 澤     開啟 張貼者 潘俞 岑 張貼 日期   Google LLC  Amphitheatre Parkway Mountain View CA  USA 不想 收到 Classroom 傳送 電子郵件 取消 訂閱']
SPAM_10=['直播 預告 利用 NanEye 快速 開發 一次性 醫療 內窺鏡 顯示 點擊 在線 查看 直播 預告 利用 NanEye 快速 開發 一次性 醫療 內窺鏡 一次性 內窺鏡 醫療 內窺鏡 市場 成? 主流 趨勢 降低 交叉感染 風險 尺寸 內窺鏡 泌尿 鏡 支氣管鏡 人體 進行 無創 手術 診斷 治療 一種 最佳 方式 一次性 內窺鏡 市場 需求 ams OSRAM 提供 NanEye 系列 微 攝像頭 模組 客戶 快速 開發出 市場 競爭力 一次性 內窺鏡 產品 本次 研討會 將會 解  醫療 內窺鏡 市場趨勢  NanEye 系列 微型 攝像頭 產品 優勢 ']

HAM_1=['發送 郵件 退回 相關 信息 發件人 emailAddress 主題 Payment account 原文 郵件 原文 包含 附件 中 時間       CST 發送到 emailAddress 系統 應答  退信 原因 發件 賬號 無權 限給 郵件 列表 發送 郵件 導致 郵件 系統 退回 解決 建議 確認 郵件 列表']
HAM_2=['hi sure get early message figure probably slip crack help many company increase sale want show something would like know available thanks carter smith ecommerce specialist emailaddress carter smith emailaddress date wed jun     cest  emailaddress emailaddress']
HAM_3=['Hi 現場 還在 佈 光纖 網線 接 Wifi AP 落伍 透過 網狀 網路 Mesh 結構 無線 AP 連線 建立 備援 架構 輕鬆 建立 單一 無線 系統 改善 角落 訊號 業界 首發 WA512GM Mesh Wifi AP 提供 IP67 導軌 式 設計 戶外 Wifi 傳輸 困擾 Womater 提供 長 距離 Wifi 方案 Tim Chen 陳德 智 Sales developing Div 展業 部 UrlText Phone   ext  MP  emailAddress']
HAM_4=['urltext']
HAM_5=['awesome thank much alex wonderful day well aaron dyke chief executive officer p   e emailaddress urltext office  washington mall  boston  something stand success move dwayne rock johnson tue feb     emailaddress write hello aaron hope great weekend track late']
HAM_6=['message summit web portal      receive send back sale order po  part c67711t7a003101 r please recheck click urltext']
HAM_7=['Dear friend 參考 黃埔 價格 用到   號 建華 USD700  陽明 USD750  中聯 基隆 USD950  中聯 臺中 高雄 USD950  臺塑 RMB6300 RMB14500 南沙 建華 USD850   價格 用到   號 中聯 基隆 USD950  價格 用到   號 中聯 臺中 高雄 USD950  價格 用到   號']
HAM_8=['正確 顯示 點選 查看 知識 學院 網站 企業 包班 企業 二代 高?會 物料 控制 倉儲 管理 操作實務 課前 導讀 影片 連結 何種 製造業 物料 倉儲 管理 工作 迄今 困擾 管理層 問題所在 倉儲 管理 做 不好 良品 廢料 損失 備料 費時 發錯 料 影響 生產 工作 料帳 不準 更是 企業 心頭 之痛 不必要 呆料 停工待料 導致 交期 延誤 庫存 管理模式 攸關 投產 供料 成敗 ERP 系統  生 部門  物管 部門  採購 部門  製造 部門 廠長 製造 現場 相關 主管  IT 部門 相關 人員 中高階 主管 深入探討 物料 管理 本質 肇因 簡捷 系統化 制度化 方法 予以 防治 消除 課程 大綱 課程內容 教學方式 物料 管理 基本概念  物料 管理 意義  物料 管理 演進  物料 管理 目標  物料 管理 常見 解決 對策  物料 管理 二大 主軸  物管 人員 應有 職責 特質  物料 管理 績效 衡量 指標 PPT 講授 案例 分享 小組討論 選用 合適 存量 管制 模式  傳統 物料 計畫 迷思  庫存 存量 管制 方式  庫存 存量 管制 方式  庫存 存量 管制 方式 實務 演練 庫存 物料 需求 計算  Time Phasing 存量 管制 方式  ABC 分析 設定 存量 管制 模式  存量 管制 模式  訂單 批量 法 By Order  複 倉法  定期 訂購 法  訂購 點法 ROP 實務 演練 訂購 點 計算 方式  存量 法  物料 需求 計畫 MRP 實務 演練 貴公司 原物料 存量 管制 方式 物料 需求 計畫 MRP  製造業 ERP 作業 流程 簡介  MRP 三大 要素 主 生產 排程 MPS 用料 清單 BOM 庫存 管理 Inventory  物料 編號 原則  BOM 建立 ECN 管理  ERP 系統 導入 步驟 成功 要訣 案例 ERP 製造業 營運 管理 效益 倉儲 管理 實施 要點  倉儲 管理 目標  倉儲 管理 改善 作法  儲位 規劃 管理  做好 倉庫 5S 管理  收 發料 作業 管理 實例 收發 料 作業 流程  退料 作業 管理 實例 退料 作業 流程  先進先出 實施 技巧  料帳 合一 關鍵因素  做好 盤點 作業  盤點 目的  盤點 種類 料帳 實地  盤點 前 工作  盤點 作業 執行 步驟  ERP 進行 盤點 作業  呆 廢料  呆料 比率 高 原因  呆料 防治 解決 對策  廢料 發生 原因  廢料 原則 主辦單位 得視 報名 情況 保留 開課日期 講師 題綱 變更 權利 最新 訊息 上網 UrlText 陳 錦河 講師 講師 簡介 現 企業 競爭力 提升 顧問 專業 講師 北京大學 清華大學 四川大學 特聘 講師 安傳 電子 股份 有限公司 品管 課長 三光 惟達 股份 有限公司 生 專員 工業 技術 研究院 電子 物管 主管 華智 資訊 管理 顧問 有限公司 ERP 專案 經理 總經理 工業 技術 研究院 量測 中心 品保 課長 鼎沛 股份 有限公司 協理 副總經理 蘇州 華傑 電子 有限公司 總經理 康可 電子 股份 有限公司 副總經理 天路 管理 顧問 有限公司 總經理 廈門 福友 企 管理 顧問 有限公司 副總經理 榮 譽 榮獲 臺灣 經濟部 品質 推廣 服務獎 證 品管 技術師 合格  ISO  實驗室 品質 系統 認證 主任 評審員 考試合格 中階 幹部 管理 技能 MTP 合格 講師 專 長 企業 體質 再造 流程 優化 企業 規範化 管理 生產 管理系統 物料 管理系統 品質 管理系統 5S 目視 管理 IE 工作 改善 ERP 系統 設計規劃 目標 管理 績效考覈 人力資源 管理系統 中 基層幹部 管理 能力 提升 訓練 臺灣 區 電機 電子 工業 同業公會 品質 技術諮詢 顧問 企業 化 小組 召集人 臺灣 經濟部 中小企業 榮譽 指導員 職訓局 職業 訓練 研究 發展 中心 企業 訓練 輔導團 顧問 區域 上課 日期 上課 時間 地址 線上 報名 臺北場       鼎新 臺北 總公司 新北市 新店區 中興路 一段  號 臺中場       鼎新 臺中 分公司 臺中市 大里區 中興路 一段  號 課程 費用   元整 包含 稅 講義 中餐 課程 優惠 方案 線上 報名 優惠 方案 一律 享有  折 優惠 早鳥票 開課 一個月 前 報名 享有  折 優惠 團體  享有  折 優惠 Ps 人工 報名 恕 提供 優惠 課程 洽詢    分機  分機  知識 學院 網站 企業 包班 企業 二代 高?會 Facebook YouTube 客服 信箱 隱私權 聲明 取消 訂閱 鼎新 電腦 版權所有 Copyright emojiPic Data Systems Consulting Co Ltd All rights reserved 不想 收到 電子報 點選 入口 取消 訂閱']
HAM_9=['airplus engineering co ltd    f   e mail emailaddress website urltext']
HAM_10=['dear tram plz provide debit note feb within tmr thanks advance wai yan naing lym purchasing every import shipment need provide co certificate origin shipment show make country cargo']



## single token inference 
### Classification for each token and statistic 

In [7]:
model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}
predict_list = []
score_list = []
sm_score_list = []
test_token = edm_1    # set which token

print(test_token[0].split())
for token_list in test_token[0].split():
    tokenized_input = tokenizer(token_list,
                                max_length=20,
                                truncation=True,
                                return_tensors="pt")
    with torch.no_grad():
        input_items = {key: val.to(device) for key, val in tokenized_input.items()}
#         del input_items['token_type_ids'] ## bart不需要這個

        outputs = model(**input_items)
        prediction = outputs.logits.argmax(dim=-1)

        prediction = int(prediction)
        sm = softmax(outputs.logits[0])

        predict_list.append(prediction)
        score_list.append(outputs.logits[0][prediction].item())
        sm_score_list.append(sm[prediction].item())


# dic = {"abc":0}
# for key in predict_list:
#     dic[key] = dic.get(key, 0) + 1
# print(dic)

print(predict_list)
print(score_list)
print(sm_score_list)
for num, i in enumerate(predict_list):
    if i == 0:   # choose SPAM
        if test_token[0].split()[num] not in edm_dict.keys():
            edm_dict[str(test_token[0].split()[num])] = 1
        else:
            edm_dict[str(test_token[0].split()[num])] += 1
print("--------")
print(edm_dict)

['新', '登入', '資訊', '確認', 'FedEx', '帳號', 'XXXXX4045', 'Alice', '感謝', '支持', 'FedEx', '登入', '整合', '計畫', 'UrlText', '登入', '資訊', '成功', '登入', 'FedEx', '酬賓', '廣場', '平臺', '這讓', '得以', 'UrlText', '用戶', '識別碼', 'FedEx', '酬賓', '廣場', '安排', '管理', '到府', '取件', '管理', '線上', '帳單', '視', '查詢', '特定', '運送', '時間', '登入', 'FedEx', '酬賓', '廣場', '查看', '紅利', '積分', '回饋', '獎品', '更多', '功能', '如需', '進一步', 'UrlText', '帳號', '登入', 'FedEx', '酬賓', '廣場', '網站', '請參閱', '保護', '帳號', '顯示', '碼', 'FedEx', '帳號', '碼', 'UrlText', '登入', '帳號', '註冊', 'FedEx', '酬賓', '廣場', 'FedEx', '帳號', '受', 'FedEx', '酬賓', '廣場', '活動', '條款', '細則', '關注', 'FEDEX', '活動', '訊息', '通知', '發送至', 'emailAddress', 'FedEx', '不定期', '寄發', '活動', '訊息', '至此', '信箱', '取消', '訂閱', '更新', '資料', '如本', '電子郵件', '涉及', '個人資料', '無意', '接受', '類似', '訊息', '按此', '回覆', '妥善處理', '荷蘭商', '聯邦快遞', '國際', '股份', '有限公司', '臺灣', '分公司', '臺北市', '中山北路', '二段', '號', '樓', 'emojiPic', 'FedEx', '本信件', '受', '美國', '國際', '著作權', '商標法', '保護', '參考', '隱私權', '聲明', '保留', '權利', '瀏覽', '網路版']


/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0]
[5.311371326446533, 3.574253559112549, 2.0248451232910156, 1.6400669813156128, 4.155544757843018, 1.9316411018371582, 2.539412498474121, 6.331620216369629, 1.7796704769134521, 3.784069538116455, 4.155544757843018, 3.574253559112549, 5.333207607269287, 3.4603688716888428, 3.814969301223755, 3.574253559112549, 2.0248451232910156, 4.109175205230713, 3.574253559112549, 4.155544757843018, 6.1882219314575195, 5.217811584472656, 2.2391433715820312, 5.76027250289917, 2.059264659881592, 3.814969301223755, 2.969763994216919, 2.628120183944702, 4.155544757843018, 6.188221

## EDM statistic 

In [8]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2}
model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('right_edm.csv')[:10]    # change the value up to the colab limit
    # rows = csv.reader(csvfile)
for i, token_list in enumerate(csvfile['context']):
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []
    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == 1:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))
        

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'免運': 70, '訂閱': 14, '取消': 10, '限時': 6, '線上': 5, '購物': 4, '點選': 4, '洗衣機': 3, '按此': 3, '閱讀': 2, '銷售': 2, '顯示': 2, 'Subscribe': 2, 'Unsubscribe': 2, '誤判為': 2, '消費': 2, '提供': 2, '顯示器': 2, '請點此': 1, '本活動': 1, '去除': 1, '攝影機': 1, '資生堂百優': 1, '外貿協會': 1, '推出': 1, '轉介': 1, 'X10': 1, 'Panasonic': 1, 'XB12': 1, '抽取': 1, '星級飯店': 1, '曬衣架': 1, '刪除': 1, '電腦病毒': 1}


# HAM statistic

In [9]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2}
model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('right_ham.csv')[:10]  # change the value up to the colab limit
    # rows = csv.reader(csvfile)
for i, token_list in enumerate(csvfile['context']):
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []
    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == 0:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'求職者': 22, '待遇': 14, '交易': 13, '信件': 6, 'Windows': 6, '回覆': 5, '信用卡': 5, '股份': 5, '收件人': 5, 'sender': 5, '謝謝': 5, '待業': 5, '個人資料': 4, '請款': 4, '收到': 4, '寄件人': 4, 'virus': 4, '寫': 4, 'email': 4, '登入': 3, '新臺幣': 3, '費用': 3, '信用': 3, 'recipient': 3, '檢視': 3, '持有': 3, '接待': 3, '簽約': 2, '損害': 2, 'please': 2, 'notify': 2, 'communications': 2, '薪資': 2, '主任': 2, '作業系統': 2, 'XP': 2, '收取': 2, '提醒您': 2, '客運': 2, 'HKC110292': 2, 'subject': 2, 'pm': 2, 'shall': 2, 'customer': 2, 'statement': 2, '主旨': 1, '寄件': 1, '21KK279981352': 1, '產品編號': 1, '產品名稱': 1, '訂購': 1, '若需': 1, '旅客': 1, '客戶': 1, 'NT': 1, '如非': 1, '扣繳': 1, '需為': 1, '新增': 1, '並請': 1, 'distribution': 1, 'using': 1, 'damages': 1, 'transmitted': 1, '行政助理': 1, '基金會': 1, '總務組長': 1, 'Android': 1, '接受': 1, '小客車': 1, '食物': 1, '入帳日': 1, '1ESHKITT011555': 1, 'notification': 1, 'transaction': 1, 'result': 1, 'message': 1, 'send': 1, 'PM': 1, '請於': 1, '寄件者': 1, '涉及': 1, '有價證券': 1, 'addressee': 1, 'may': 1, 'contain': 1, 'received': 1, 'error': 1, 'pro

# SPAM statistic

In [10]:
import pandas as pd
import operator
class_number = {'SPAM':0, 'EDM':1, 'HAM':2}
model.eval()

softmax=torch.nn.Softmax()
# predict_list = []
# score_list = []
# sm_score_list = []

edm_dict = {}

# with open('right_edm.csv', newline='', encoding='utf-8') as csvfile:
csvfile = pd.read_csv('right_spam.csv')[:10]   # change the value up to the colab limit
    # rows = csv.reader(csvfile)
for i, token_list in enumerate(csvfile['context']):
    # print(token_list)
    token_list = token_list.split()
    predict_list = []
    score_list = []
    sm_score_list = []
    for token in token_list:

        tokenized_input = tokenizer(token,
                                    max_length=20,
                                    truncation=True,
                                    return_tensors="pt")
        with torch.no_grad():
            input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    #         del input_items['token_type_ids'] ## bart不需要這個

            outputs = model(**input_items)
            prediction = outputs.logits.argmax(dim=-1)

            prediction = int(prediction)
            sm = softmax(outputs.logits[0])

            predict_list.append(prediction)
            score_list.append(outputs.logits[0][prediction].item())
            sm_score_list.append(sm[prediction].item())


    dic = {}
    for key in predict_list:
        dic[key] = dic.get(key, 0) + 1
    # print(dic)

    # print(predict_list)
    # print(score_list)
    # print(sm_score_list)
    for num, j in enumerate(predict_list):
        if j == 0:
            # print(token_list[num])
            if token_list[num] not in edm_dict.keys():
                edm_dict[str(token_list[num])] = 1
            else:
                edm_dict[str(token_list[num])] += 1
    # print("--------")

print(dict(sorted(edm_dict.items(), key=operator.itemgetter(1),reverse=True)))

/home/soc507/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'思維導圖': 70, 'UrlText': 67, 'cnn': 58, 'emailAddress': 46, '案例': 35, '分析': 35, '數據': 32, '管理': 31, '表達': 30, '課程': 28, 'old': 27, '工作': 26, '邏輯': 24, 'Excel': 24, 'videoplayer': 23, '財務': 22, 'like': 21, '金字塔': 21, 'I': 21, 'gabeira': 21, '筆記': 20, 'PPT': 20, 'wave': 20, 'year': 18, '講': 18, '圖表': 18, 'm': 17, 'big': 17, '企業': 17, 'Y': 17, 'say': 16, 'He': 16, 'Smeeth': 16, 'boys': 16, 'fellows': 16, 'going': 16, 'Heart': 16, 'Sheldy': 16, '思維': 15, '做': 15, 'videopinner': 15, '結構': 14, 'know': 14, 'type': 14, 'playerinstance': 14, 'turn': 13, '原理': 13, '講師': 13, 'get': 13, 'currentvideocollection': 13, 'mobilepinnedview': 13, 'world': 12, '清晰': 12, '基礎': 12, '中': 12, '設計': 12, '成本': 12, 'men': 12, 'tell': 12, 'nazar': 12, 'way': 11, '高效': 11, '認證': 11, '學士': 11, '表': 11, 'sport': 11, 'use': 11, 'jquery': 11, '控制': 11, 'make': 10, 'give': 10, 'go': 10, 'come': 10, '構建': 10, 'C': 10, '方法': 10, '繪製': 10, '關鍵詞': 10, '軟件': 10, '演示': 10, '技巧': 10, '圖': 10, '預算': 10, '透視': 10, 'dinner': 10, 

## token clissifier

In [11]:
test_num = 10
test_list = pd.read_csv('right_spam.csv')
# subject = test_list[test_num]['subject']
print(test_list['context'][10])
context = test_list['context'][test_num]
label = test_list['ref'][test_num]

產品 中試 管理 樣品 量產 Product pilot management sample mass production    上海    深圳     北京  元 課程 背景 企業 提供 研發 管理 諮詢服務 過程 中 發現 企業 新 產品開發 樣機 量產 過程 中 產品化 過程 着 新品 中試 中試 時間 短 製造 部門 戲稱 研發 新品 三無 產品 生產 文件 工裝 生產 現場 沒人管 轉產 標準 研發 想 快點 轉產 生產 產品 不願 接收 希望 研發 解決 轉過來 市場 催得 急 被迫 接收 長此以往 導致 研發 生產 矛盾激化 企業 成立 中試 部門 希望 中試 階段 產品質量 解決掉 中試 位與 運作 困惑 發生 質量 進度 衝突 時 取捨 平衡 研發 製造 矛盾轉化 研發 中試 中試 生產 矛盾 中試 成 矛盾 集散 中心 市場 壓力 並不因 中試 減少 中試 哪些方面 努力 產品 質量 進度 中試 業務 面向 研發 面向 製造 兼而有之 量產 發現 產品 製造 性差 成品率 低 返工 影響 發貨 產品 生產 後還 發生 設計變更 產品 客戶 手中 還冒出 各種各樣 研發 人員 救火 課程 多年 實踐 長期 研發 諮詢 積累 一套 理論 實踐 相結合 操作 方法 配以 案例 指導 研發 試產 製造 部門 主管 高效 產品 樣品 走向 量產 參加 對象 企業 CEO 總經理 研發 總經理 副總 測試 部經理 中試 試產 部經理 製造 部經理 工藝 工程 部經理 質量 部經理 項目 經理 產品 經理 高級 製造 工程師 培訓 收益 業界 公司 發展 階段 產品 中試 管理模式 實踐 面向 製造 系統 產品設計 DFM 方法 實施 過程 面向 生產 測試 產品設計 DFT 方法 實施 過程 面向 製造 系統 新 產品 驗證 過程 方法 質量 標準 前提 縮短 產品 試製 週期 方法 技巧 建立 樣品 量產 管理機制 課程 大綱 案例 研討 樣品 量產 概述 企業 追求 技術 樣品 產品 商品 研發 製造 矛盾  製造 系統 面對 研發 三無 產品  研發 面對 製造 系統 越來越 高 門檻 研發 製造矛盾 激化 中試 成爲 中試 位與 發展  研發 RD 中試 D P 生產 P 關係  中試 使命  中試 定位  中試 發展 大而全 b 專業化 分工 

In [12]:
tokenized_input = tokenizer(context,
                            max_length=512,
                            truncation=True,
                            return_tensors="pt")
class_number = {'SPAM':0, 'EDM':1, 'HAM':2}
model.eval()
with torch.no_grad():
    input_items = {key: val.to(device) for key, val in tokenized_input.items()}
    del input_items['token_type_ids'] ## bart不需要這個
    
    outputs = model(**input_items)
    prediction = outputs.logits.argmax(dim=-1)
    print(type(int(prediction)))
    
    
    # print('主旨: ', subjzect)
    print('內文: ', context)
    print('label: ', class_number[label], label)
    print('predict: ', int(prediction), list(class_number.keys())[list(class_number.values()).index(int(prediction))])
    
    # if int(prediction) == 0:
    #     print('predict: ham')
    # else:
    #     print('predict: spam')

<class 'int'>
內文:  產品 中試 管理 樣品 量產 Product pilot management sample mass production    上海    深圳     北京  元 課程 背景 企業 提供 研發 管理 諮詢服務 過程 中 發現 企業 新 產品開發 樣機 量產 過程 中 產品化 過程 着 新品 中試 中試 時間 短 製造 部門 戲稱 研發 新品 三無 產品 生產 文件 工裝 生產 現場 沒人管 轉產 標準 研發 想 快點 轉產 生產 產品 不願 接收 希望 研發 解決 轉過來 市場 催得 急 被迫 接收 長此以往 導致 研發 生產 矛盾激化 企業 成立 中試 部門 希望 中試 階段 產品質量 解決掉 中試 位與 運作 困惑 發生 質量 進度 衝突 時 取捨 平衡 研發 製造 矛盾轉化 研發 中試 中試 生產 矛盾 中試 成 矛盾 集散 中心 市場 壓力 並不因 中試 減少 中試 哪些方面 努力 產品 質量 進度 中試 業務 面向 研發 面向 製造 兼而有之 量產 發現 產品 製造 性差 成品率 低 返工 影響 發貨 產品 生產 後還 發生 設計變更 產品 客戶 手中 還冒出 各種各樣 研發 人員 救火 課程 多年 實踐 長期 研發 諮詢 積累 一套 理論 實踐 相結合 操作 方法 配以 案例 指導 研發 試產 製造 部門 主管 高效 產品 樣品 走向 量產 參加 對象 企業 CEO 總經理 研發 總經理 副總 測試 部經理 中試 試產 部經理 製造 部經理 工藝 工程 部經理 質量 部經理 項目 經理 產品 經理 高級 製造 工程師 培訓 收益 業界 公司 發展 階段 產品 中試 管理模式 實踐 面向 製造 系統 產品設計 DFM 方法 實施 過程 面向 生產 測試 產品設計 DFT 方法 實施 過程 面向 製造 系統 新 產品 驗證 過程 方法 質量 標準 前提 縮短 產品 試製 週期 方法 技巧 建立 樣品 量產 管理機制 課程 大綱 案例 研討 樣品 量產 概述 企業 追求 技術 樣品 產品 商品 研發 製造 矛盾  製造 系統 面對 研發 三無 產品  研發 面對 製造 系統 越來越 高 門檻 研發 製造矛盾 激化 中試 成爲 中試 位與 發展  研發 RD 中試 D P 生產 P 關係  中試 使命  中試 定位  